# Feature addition and reduction

As seen in the inferential statistics notebook, there are a lot of multicollinearities going on among our variables. In this notebook I will try to delete some redundant features, combine some highly correlated features, and create some features using our existing data that I think might be helpful for our prediction.

In [79]:
#Load necessary modules
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from datetime import timedelta
import numpy as np
from tqdm import tqdm

First, we load our dataset to see what features we do have.

In [80]:
df = pd.read_csv(r'C:\Users\songs\Desktop\Springboard Files\Springboard-Files\Capstone 2\data\Interim\train1.csv',index_col=0)
df.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,MarkDown5,CPI,Unemployment,Size,Type_A,Type_B,Type_C,log_revenue,IsMarkDown,Type_barplot
0,26,92,2011-08-26,87235.57,False,61.10,3.796,0.00,0.0,0.0,...,0.00,136.213613,7.767,152513,1,0,0,11.431992,False,A
1,34,22,2011-03-25,5945.97,False,53.11,3.480,0.00,0.0,0.0,...,0.00,128.616064,10.398,158114,1,0,0,9.299807,False,A
2,21,28,2010-12-03,1219.89,False,50.43,2.708,0.00,0.0,0.0,...,0.00,211.265543,8.163,140167,0,1,0,8.733889,False,B
3,8,9,2010-09-17,11972.71,False,75.32,2.582,0.00,0.0,0.0,...,0.00,214.878556,6.315,155078,1,0,0,9.738769,False,A
4,19,55,2012-05-18,8271.82,False,58.81,4.029,12613.98,0.0,11.5,...,3600.79,138.106581,8.150,203819,1,0,0,9.492640,False,A


In [81]:
df.describe()

,Store,Dept,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,Type_A,Type_B,Type_C,log_revenue
count,282450.000000,282450.000000,282450.000000,282450.000000,282450.000000,282450.000000,282450.000000,282450.000000,282450.000000,282450.00000,282450.000000,282450.000000,282450.000000,282450.000000,282450.000000,282450.000000,282450.000000
mean,22.193146,44.286274,15983.503944,60.113598,3.360301,2578.777540,872.126294,459.273032,1077.245617,1659.48530,171.207961,7.968076,136729.826904,0.511800,0.387371,0.100828,9.571147
std,12.782156,30.503610,22661.098250,18.446505,0.458603,6023.733269,5077.429516,5475.263454,3874.121495,4252.38421,39.160786,1.868035,61002.286891,0.499862,0.487150,0.301102,0.822145
min,1.000000,1.000000,-1750.000000,-2.060000,2.472000,0.000000,-265.760000,-29.100000,0.000000,0.00000,126.064000,3.879000,34875.000000,0.000000,0.000000,0.000000,8.083310
25%,11.000000,18.000000,2079.337500,46.780000,2.932000,0.000000,0.000000,0.000000,0.000000,0.00000,132.022667,6.891000,93638.000000,0.000000,0.000000,0.000000,8.863514
50%,22.000000,38.000000,7616.620000,62.150000,3.452000,0.000000,0.000000,0.000000,0.000000,0.00000,182.350989,7.866000,140167.000000,1.000000,0.000000,0.000000,9.441973
75%,33.000000,74.000000,20245.812500,74.290000,3.737000,2789.030000,1.910000,4.420000,423.220000,2152.40000,212.464799,8.572000,202505.000000,1.000000,1.000000,0.000000,10.136017
max,45.000000,99.000000,693099.360000,100.140000,4.468000,88646.760000,104519.540000,141630.610000,67474.850000,108519.28000,227.232807,14.313000,219622.000000,1.000000,1.000000,1.000000,13.456102


In [82]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282450 entries, 0 to 282450
Data columns (total 21 columns):
Store           282450 non-null int64
Dept            282450 non-null int64
Date            282450 non-null object
Weekly_Sales    282450 non-null float64
IsHoliday       282450 non-null bool
Temperature     282450 non-null float64
Fuel_Price      282450 non-null float64
MarkDown1       282450 non-null float64
MarkDown2       282450 non-null float64
MarkDown3       282450 non-null float64
MarkDown4       282450 non-null float64
MarkDown5       282450 non-null float64
CPI             282450 non-null float64
Unemployment    282450 non-null float64
Size            282450 non-null int64
Type_A          282450 non-null int64
Type_B          282450 non-null int64
Type_C          282450 non-null int64
log_revenue     282450 non-null float64
IsMarkDown      282450 non-null bool
Type_barplot    282450 non-null object
dtypes: bool(2), float64(11), int64(6), object(2)
memory usage: 43.6+

## Combine the Markdown Columns

As seen in the EDA notebook, the markdown columns are highly correlated with each other and are a large source of multicollinearity. By combining them into one "Total_Markdown" column, we reduce collinearity. We can also eliminate the IsMarkDown column, since a total markdown of 0 would mean that there is no markdown that week.

In [83]:
df_totalmd = df.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','IsMarkDown'],axis=1)
df_totalmd['Total_MarkDown'] = df[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].sum(axis=1)
df_totalmd.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_A,Type_B,Type_C,log_revenue,Type_barplot,Total_MarkDown
0,26,92,2011-08-26,87235.57,False,61.10,3.796,136.213613,7.767,152513,1,0,0,11.431992,A,0.00
1,34,22,2011-03-25,5945.97,False,53.11,3.480,128.616064,10.398,158114,1,0,0,9.299807,A,0.00
2,21,28,2010-12-03,1219.89,False,50.43,2.708,211.265543,8.163,140167,0,1,0,8.733889,B,0.00
3,8,9,2010-09-17,11972.71,False,75.32,2.582,214.878556,6.315,155078,1,0,0,9.738769,A,0.00
4,19,55,2012-05-18,8271.82,False,58.81,4.029,138.106581,8.150,203819,1,0,0,9.492640,A,17931.55


## Delete Type_barplot

I had originally created this column for easier plotting in seaborn, but now that we're not doing plotting, we can delete the column. 

Type of store is a categorical variable, and when doing regression, it's important to have a baseline for categorical variables so that we can interpret the effect of other options in the category. I chose type B as the baseline because it is in the middle in terms of size, and it's be easy to see positive effects (Type A) or negative effects (Type C)

In [84]:
df_nob = df_totalmd.drop(['Type_barplot','Type_B'],axis=1)
df_nob.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_A,Type_C,log_revenue,Total_MarkDown
0,26,92,2011-08-26,87235.57,False,61.10,3.796,136.213613,7.767,152513,1,0,11.431992,0.00
1,34,22,2011-03-25,5945.97,False,53.11,3.480,128.616064,10.398,158114,1,0,9.299807,0.00
2,21,28,2010-12-03,1219.89,False,50.43,2.708,211.265543,8.163,140167,0,0,8.733889,0.00
3,8,9,2010-09-17,11972.71,False,75.32,2.582,214.878556,6.315,155078,1,0,9.738769,0.00
4,19,55,2012-05-18,8271.82,False,58.81,4.029,138.106581,8.150,203819,1,0,9.492640,17931.55


## Deciphering Year, Month, Week of Year, and Day

The year, month, week, and day could have an impact on the revenue made. As of now, the date column won't work with a lot of models. Thus, I'm going to parse the date column into year, month week of year, and day columns.

In [85]:
df_parsedate = df_nob
df_parsedate['Date'] = df_parsedate['Date'].astype('datetime64')
df_parsedate['Year'] = df_nob['Date'].dt.year
df_parsedate['Month'] = df_nob['Date'].dt.month
df_parsedate['Week_of_year'] = df_nob['Date'].dt.weekofyear
df_parsedate['Day'] = df_nob['Date'].dt.day
df_parsedate.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_A,Type_C,log_revenue,Total_MarkDown,Year,Month,Week_of_year,Day
0,26,92,2011-08-26,87235.57,False,61.10,3.796,136.213613,7.767,152513,1,0,11.431992,0.00,2011,8,34,26
1,34,22,2011-03-25,5945.97,False,53.11,3.480,128.616064,10.398,158114,1,0,9.299807,0.00,2011,3,12,25
2,21,28,2010-12-03,1219.89,False,50.43,2.708,211.265543,8.163,140167,0,0,8.733889,0.00,2010,12,48,3
3,8,9,2010-09-17,11972.71,False,75.32,2.582,214.878556,6.315,155078,1,0,9.738769,0.00,2010,9,37,17
4,19,55,2012-05-18,8271.82,False,58.81,4.029,138.106581,8.150,203819,1,0,9.492640,17931.55,2012,5,20,18


## Adding the Yearly Median value of each Store-Dept combination

This would provide a baseline value for each store-department combination that the models could modify for each week given the other attributes. I'm using the median instead of the mean because the mean can be biased by extreme values, and thus won't give as clear an indicator of each store-department's average performance.

In [86]:
df_groupby = df_parsedate.groupby(['Store','Dept','Year'])['Weekly_Sales'].median()
df_median = df_parsedate.merge(df_groupby, on=['Store','Dept','Year'], how='outer')
df_median.rename(columns={'Weekly_Sales_x':'Weekly_Sales','Weekly_Sales_y':'Median_Sales'}, inplace=True)
df_median.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_A,Type_C,log_revenue,Total_MarkDown,Year,Month,Week_of_year,Day,Median_Sales
0,26,92,2011-08-26,87235.57,False,61.10,3.796,136.213613,7.767,152513,1,0,11.431992,0.0,2011,8,34,26,80340.795
1,26,92,2011-09-23,80254.53,False,50.72,3.758,136.367000,7.767,152513,1,0,11.353279,0.0,2011,9,38,23,80340.795
2,26,92,2011-05-27,77715.58,False,56.74,4.034,134.767774,7.818,152513,1,0,11.323042,0.0,2011,5,21,27,80340.795
3,26,92,2011-07-01,83599.56,False,59.89,3.815,135.446800,7.767,152513,1,0,11.391769,0.0,2011,7,26,1,80340.795
4,26,92,2011-04-22,77876.10,False,33.81,3.919,134.357100,7.818,152513,1,0,11.324981,0.0,2011,4,16,22,80340.795


## Adding lagged values

The median provides a baseline for our predictions, but doesn't tell us anything about how well the specific department-store combination is doing at a specific period in time. I'm going to add a "Lagged_Value" column with the weekly revenue of the department-store combination during the week before. I'm also adding a difference from median column to give information how well the particular combination did last week relative to its median.

In [87]:
df_lagged = df_median.sort_values(['Store','Dept','Date'])
df_lagged['Lagged_Date'] = df_lagged['Date'] - timedelta(days=7)
df_lagged['Lagged_Value'] = np.nan
df_lagged['Lag_Available'] = True
for i in tqdm(range(df_lagged.shape[0])):
    if df_lagged.iloc[i]['Lagged_Date'] == df_lagged.iloc[i-1]['Date']:
        df_lagged.set_value(df_lagged.index[i],'Lagged_Value',df_lagged.iloc[i-1]['Weekly_Sales'])
    else:
        df_lagged.set_value(df_lagged.index[i],'Lagged_Value',df_lagged.iloc[i]['Median_Sales'])
        df_lagged.set_value(df_lagged.index[i],'Lag_Available',False)

  0%|                                                                                       | 0/282450 [00:00<?, ?it/s]C:\Users\songs\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':
C:\Users\songs\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  # Remove the CWD from sys.path while we load stuff.
C:\Users\songs\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  import sys
  6%|████▏                                                                    | 15979/282450 [00:07<02:03, 2153.80it/s]

KeyboardInterrupt: 

In [ ]:
df_lagged['Diff_from_Med'] = df_lagged['Lagged_Value'] - df_lagged['Median_Sales']
df_lagged = df_lagged.drop('Lagged_Date',axis=1)
df_lagged.head()

## Adding some Date variables

Some models don't work well with the dates, so I'm going to see if any dates have a larger impact than others.

From from the >$300,000 table in the EDA notebook, it appears that a signficant portion of them are made in the thanksgiving week, so we'll make a new feature called "IsThanksgiving" to distinguish those values made during Thanksgiving weeks. Looking at the average sales per week, it appears that sales are higher near Christmas, so I'll also create a new feature called "IsChristmas". After this, we could delete the Date Column.

In [ ]:
df_bigholidays = df_lagged
df_bigholidays['IsThanksgiving'] = (df_bigholidays['Date'] == '2011-11-25') | (df_bigholidays['Date'] == '2010-11-26')
df_bigholidays['IsChristmas'] = (df_bigholidays['Date'] == '2010-12-31') | (df_bigholidays['Date'] == '2011-12-30')
df_bigholidays['IsPreChristmas'] = (df_bigholidays['Date'] == '2010-12-24') | (df_bigholidays['Date'] == '2011-12-23')
df_bigholidays = df_bigholidays.drop(['Date'],axis=1)
df_bigholidays.head()

In [ ]:
df_bigholidays.info()

## Adding department 72

Looking at the EDA, it appears that department 72 has sales higher than others. I'll create an "IsDept72" Column, then.

In [ ]:
df_dept72 = df_bigholidays
df_dept72['IsDept72'] = (df_bigholidays['Dept'] == 72)
df_dept72.head()

In [91]:
for i in range(5):
    fit_model(df_totalm)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [88]:
def fit_model(df):
    #Creating Features and Target
    X = df.drop(['Weekly_Sales','log_revenue'], axis=1).values
    y = df['Weekly_Sales'].values

    #Train Test Split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)
    
    #Create the Regression model:
    linear = LinearRegression()
    linear.fit(X_train, y_train)

    #Making predictions
    y_pred = linear.predict(X_test)
    
    y_true = y_test
    
    print(mean_absolute_error(y_true, y_pred))
    
for i in range(5):
    fit_model(df_dept72)

2213.5877813856778
2215.539381385311
2230.7569323956723
2213.302786631987
2229.9675646485193


In [90]:
df_totalm = df_dept72.drop(['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5','IsMarkDown'],axis=1)
df_totalm['Total_MarkDown'] = df[['MarkDown1','MarkDown2','MarkDown3','MarkDown4','MarkDown5']].sum(axis=1)
df_totalm.head()

,Store,Dept,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,CPI,Unemployment,Size,Type_A,...,Day,Median_Sales,IsThanksgiving,IsChristmas,IsPreChristmas,Lagged_Value,Lag_Available,Diff_from_Med,IsDept72,Total_MarkDown
106604,1,1,46039.49,True,38.51,2.548,211.242170,8.106,151315,1,...,12,19239.405,False,False,False,19239.405,False,0.000,False,0.00
106610,1,1,19403.54,False,46.63,2.561,211.319643,8.106,151315,1,...,26,19239.405,False,False,False,19239.405,False,0.000,False,2688.68
106598,1,1,21827.90,False,46.50,2.625,211.350143,8.106,151315,1,...,5,19239.405,False,False,False,19403.540,True,164.135,False,0.00
106620,1,1,21043.39,False,57.79,2.667,211.380643,8.106,151315,1,...,12,19239.405,False,False,False,21827.900,True,2588.495,False,3382.65
106607,1,1,57258.43,False,62.27,2.719,210.820450,7.808,151315,1,...,2,19239.405,False,False,False,19239.405,False,0.000,False,42674.66


In [77]:
df_dept72.to_csv(r'C:\Users\songs\Desktop\Springboard Files\Springboard-Files\Capstone 2\data\Interim\train_all_features.csv')